In [ ]:
import sys,os

In [ ]:
sys.path.append("/opt/notebooks/GPU-GWAS/")
os.chdir("/opt/notebooks/GPU-GWAS/")

In [ ]:
import argparse
import time
from collections import defaultdict

import cupy as cp
import cudf
import pandas as pd
import rmm

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import gpugwas.io as gwasio
import gpugwas.algorithms as algos
import gpugwas.viz as viz
import gpugwas.dataprep as dp
import gpugwas.runner as runner
from gpugwas.vizb import show_qq_plot, show_manhattan_plot

In [ ]:
vcf_path='./data/test.vcf'
annotation_path='./data/1kg_annotations.txt'

In [ ]:
# Initialize Memory Pool to 10GB
cudf.set_allocator(pool=True, initial_pool_size=1e10)
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

## Load data into dataframes

In [ ]:
# Load data
vcf_df, feature_mapping = gwasio.load_vcf(vcf_path, info_keys=[], format_keys=["GT"])
#vcf_df = cudf.io.parquet.read_parquet("/data/1000-genomes/hail-dataset/1kg_full_jdaw_v2.pqt")
ann_df = gwasio.load_annotations(annotation_path)
print(vcf_df)
print("==")
print(ann_df)

## Generate phenotype dataframe by merging vcf and annotation DF

In [ ]:
phenotypes_df, features = dp.create_phenotype_df(vcf_df, ann_df, ['CaffeineConsumption','isFemale','SuperPopulation'], "call_GT",
                                       vcf_sample_col="sample", ann_sample_col="Sample")

## Run PCA on phenotype matrix

In [ ]:
# Run PCA on phenotype dataframe
phenotypes_df = algos.PCA_concat(phenotypes_df, 2)
print(phenotypes_df)

In [ ]:
colors = {'AFR':'red', 'AMR':'green', 'EAS':'blue', 'EUR':'yellow', 'SAS':'purple'}
from matplotlib.lines import Line2D
plt.scatter(phenotypes_df.PC0.to_array(), phenotypes_df.PC1.to_array(), 
            c=phenotypes_df.SuperPopulation.to_pandas().map(colors).values, s=9)
legend_elements = [Line2D([0], [0], marker='o', color='w', label=key, 
                          markerfacecolor=value) for key, value in colors.items()]
plt.legend(handles=legend_elements)

## Run GWAS with linear regression for each independent variant

In [ ]:
# Fit linear regression model for each variant feature
print("Fitting linear regression model")

df = runner.run_gwas(phenotypes_df, 'CaffeineConsumption', features, algos.cuml_LinearReg, add_cols=['PC0', 'PC1'])
print(df)

In [ ]:
plt.hist(-np.log(df["p_value"].to_array()), bins = np.linspace(0,1,100));

In [ ]:
df.drop(columns="chrom", inplace=True)
g_feature_mapping = cudf.DataFrame(feature_mapping[["feature_id", "pos", "chrom"]])
df = df.merge(g_feature_mapping, how="inner", left_on=["feature"], right_on=["feature_id"])
df.chrom = df.chrom.astype("int64")

In [ ]:
show_manhattan_plot(result, 'chrom', 'pos', 'p_value')

In [ ]:
a = df["p_value"].to_array()
a.sort()
expect_p = np.linspace(0, 1, len(a))

In [ ]:
df["e_value"] = np.linspace(0, 1, len(a))
df["p_s_value"] = a
show_qq_plot(df, 'e_value', 'p_s_value', x_max=3, y_max=3)

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show

output_notebook()

plot = figure()
plot.circle(-np.log10(expect_p+1e-10), -np.log10(a))

handle = show(plot, notebook_handle=True)

# Update the plot title in the earlier cell
plot.title.text = "qqplot"
push_notebook(handle=handle)

In [ ]:
!wget https://www.broadinstitute.org/files/shared/diabetes/scandinavs/DGI_chr3_pvals.txt

In [ ]:
pvals = []
with open('DGI_chr3_pvals.txt') as f:
    for r in f:
        r = r.strip()
        if r == 'PVAL':
            continue
        pvals.append(float(r))
pvals = np.array(pvals)

In [ ]:
pvals.sort()
expect_p = np.linspace(0, 1, len(pvals))

In [ ]:
plt.plot(-np.log10(expect_p), -np.log10(pvals), '.')
plt.plot([0,5],[0,5])

In [ ]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
f#rom bokeh.models import Range1d

output_notebook()


plot = figure(plot_width=300, plot_height=300, 
              y_range=(0,5),
              x_range=(0,5))
plot.circle(-np.log10(expect_p+1e-10), -np.log10(pvals))
plot.line([0,5],[0,5])

handle = show(plot, notebook_handle=True)

# Update the plot title in the earlier cell
plot.title.text = "qqplot"
push_notebook(handle=handle)

In [ ]:
pvals